In [1]:
# SUBMIT to Kaggle

In [2]:
from datetime import datetime
import pandas as pd
import numpy as np
import os
import xgboost as xgb
import zipfile

In [3]:
model_name = 'baseline.depth2.trn-1.80.dev-1.83.xgb'

tst = pd.read_csv('data/baseline.feature_engineer.tst.csv')
bst = xgb.Booster({'nthread': 4})
bst.load_model('model/{}'.format(model_name))

In [4]:
TST = './data/test_ver2.csv'
target_cols = ['ind_ahor_fin_ult1', 'ind_aval_fin_ult1', 'ind_cco_fin_ult1', 'ind_cder_fin_ult1',
               'ind_cno_fin_ult1', 'ind_ctju_fin_ult1', 'ind_ctma_fin_ult1', 'ind_ctop_fin_ult1',
               'ind_ctpp_fin_ult1', 'ind_deco_fin_ult1', 'ind_deme_fin_ult1', 'ind_dela_fin_ult1',
               'ind_ecue_fin_ult1', 'ind_fond_fin_ult1', 'ind_hip_fin_ult1', 'ind_plan_fin_ult1',
               'ind_pres_fin_ult1', 'ind_reca_fin_ult1', 'ind_tjcr_fin_ult1', 'ind_valo_fin_ult1',
               'ind_viv_fin_ult1', 'ind_nomina_ult1', 'ind_nom_pens_ult1', 'ind_recibo_ult1']
rem_targets = [23, 22, 2, 21, 18, 17, 4, 11, 12, 9, 6, 13, 7, 19, 8]

In [5]:
dtst = xgb.DMatrix(tst)
preds = bst.predict(dtst)
preds = np.fliplr(np.argsort(preds, axis=1))

In [6]:
print('# Generating a submission..')
submit_cols = [target_cols[i] for i, col in enumerate(target_cols) if i in rem_targets]

final_preds = []
for pred in preds:
    top_products = []
    for i, product in enumerate(pred):
        top_products.append(submit_cols[product])
        if i == 6:
            break
    final_preds.append(' '.join(top_products))

t_index = pd.read_csv(TST, usecols=['ncodpers'])
test_id = t_index['ncodpers']
out_df = pd.DataFrame({'ncodpers': test_id, 'added_products': final_preds})
file_name = 'output/' + datetime.now().strftime("{}_%Y%m%d.%H%M".format(model_name)) + '.csv'
out_df.to_csv(file_name, index=False)
with zipfile.ZipFile(file_name + '.zip', "w", compression=zipfile.ZIP_DEFLATED) as zf:
    zf.write(file_name)

# Generating a submission..


## score and ranking for baseline model 

| model | public | private | rank |
| -- | :-: | :-: | :-: |
| baseline.xgb | 0.016275 | 0.0165246 | 1304

### how can we improve!?
- read kernels/discussions to get ideas
- error analysis